In [268]:
from bs4 import BeautifulSoup
import urllib
import json
import csv
import requests

In [41]:
MEDIUM = "https://medium.com/"

In [341]:
def clean_json_response(response):
    return json.loads(response.text.split('while(1);</x>')[1])

In [3]:
url = "https://medium.com/_/api/users/af97a6a078cf/profile/stream?limit=100&source=followers"
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

In [4]:
req = urllib.request.Request(url, headers=headers)

In [5]:
with urllib.request.urlopen(req) as response:
    the_page = response.read()

In [6]:
s = (str(the_page).split("</x>")[1])[:-1]

In [7]:
from ast import literal_eval
import re
s = literal_eval("'%s'" % s)
# re.sub(r'http\S+', "", s)

In [161]:
j =json.loads(s)

In [9]:
[(d['name'], d['postCount']) for d in j['payload']['userMeta']['authorTags']]

[('Politics', 261302),
 ('Marketing', 171362),
 ('Writing', 169430),
 ('Humor', 80623),
 ('Books', 69728),
 ('Creativity', 66016),
 ('Environment', 49420),
 ('Climate Change', 39734),
 ('College', 32550),
 ('Interview', 28916),
 ('Satire', 22657),
 ('Reading', 22609),
 ('Creative Writing', 22563),
 ('Comedy', 22205),
 ('Writing Tips', 14114),
 ('Publishing', 12506),
 ('Flash Fiction', 10560),
 ('Essay', 10290),
 ('Activism', 9644),
 ('Grief', 8670),
 ('Nonfiction', 8151),
 ('Poverty', 7199),
 ('Authors', 5971),
 ('Novel', 5902),
 ('Capitalism', 5100),
 ('Protest', 5041),
 ('Creative Process', 5024),
 ('Florida', 4513),
 ('Writing Life', 4287),
 ('Fishing', 4032),
 ('Classics', 2789),
 ('Pitching', 2596),
 ('Editing', 2478),
 ('Personal Essay', 2008),
 ('Fathers', 1950),
 ('Lawsuit', 1626),
 ('Hurricane', 1358),
 ('Readinglist', 1277),
 ('MFA', 1107),
 ('Writing Advice', 238),
 ('Hurricane Florence', 99)]

In [10]:
f = j['payload']['streamItems'][1:]

In [11]:
followers = [u['userPreview']['userId'] for u in f]
len(followers)

25

In [168]:
import urllib.parse

def parseResponse(url, args):
    params = urllib.parse.urlencode(args)
    
    if len(args) != 0:
        url = url + '?%s' % params
    
    req = urllib.request.Request(url, headers=headers)
    
    with urllib.request.urlopen(req) as response:
        the_page = response.read()
        
    s = (str(the_page).split("</x>")[1])[:-1]
    s = literal_eval("'%s'" % s)
    
    j = json.loads(s)
    
    return j

In [343]:
def getFollowers(userId):
    print("Fetching user followers...")
    
    followers = set([])
    limit = 20
    hard_cap = 100
    
    url = "https://medium.com/_/api/users/" + userId + "/profile/stream?limit="+str(limit)+"&source=followers"
    response = requests.get(url)
    
    j = clean_json_response(response)
    j = j['payload']
    
    total = j['references']['SocialStats'][userId]['usersFollowedByCount']
    
    count = 0
    print("total = " + str(total))
    
    while 'next' in j['paging'] and limit*count <= min(total, hard_cap):
        
        # add new users to followers set
        f = j['streamItems'][1:]
        f = [u['userPreview']['userId'] for u in f]
        for u in f: followers.add(u)
        
        # end of pagination check
        if 'to' not in j['paging']['next']: 
            break
        
        # find next page from response
        next_url = j['paging']['path']
        next_params = j['paging']['next']
        
        # parse next page
        response = requests.get(next_url, next_params)
        j = clean_json_response(response)
        
        if 'payload' not in j: break
        
        j = j['payload']
        
        count += 1
    
    return list(followers)

In [344]:
def getFollowing(userId):
    print("Fetching user following...")
    
    following = set([])
    limit = 20
    hard_cap = 100
    
    url = "https://medium.com/_/api/users/" + userId + "/profile/stream?limit="+str(limit)+"&source=following"
    response = requests.get(url)
    
    j = clean_json_response(response)
    j = j['payload']
    
    total = j['references']['SocialStats'][userId]['usersFollowedCount']
    
    count = 0
    print("total = " + str(total))
    
    while 'next' in j['paging'] and limit*count <= min(total, hard_cap):
        
        # add new users to followers set
        f = j['streamItems'][1:]
        f = [u['userPreview']['userId'] for u in f]
        for u in f: following.add(u)
        
        # end of pagination check
        if 'to' not in j['paging']['next']: 
            break
        
        # find next page from response
        next_url = j['paging']['path']
        next_params = j['paging']['next']
        
        # parse next page
        response = requests.get(next_url, next_params)
        j = clean_json_response(response)
        
        if 'payload' not in j: break
        
        j = j['payload']
        
        count += 1
    
    return list(following)

In [236]:
def parsePostsPayload(payload, userId):
    if 'references' not in payload or 'Post' not in payload['references']: return []
    
    postIds = [k for k in payload['references']['Post']]
    
    posts = [] # post objects
    for pid in postIds:
        postObj = payload["references"]["Post"][pid]
        
        # filter out responses from claps list            
        if postObj['virtuals']['readingTime'] < 1: continue
        
        if len(postObj['virtuals']['tags']) == 0 and len(postObj['virtuals']['subtitle']) == 0:
            continue
        
        newObj = {'id': pid}
        
        newObj['clapCount'] = postObj['virtuals']['totalClapCount']
        newObj['wordCount'] = postObj['virtuals']['wordCount']
        newObj['readingTime'] = postObj['virtuals']['readingTime']
        newObj['tags'] = [tag['name'] for tag in postObj['virtuals']['tags']]
        newObj['title'] = postObj['title']
        newObj['userId'] = postObj['creatorId']
        newObj['time'] = postObj['latestPublishedAt']
        newObj['subtitle'] = postObj['virtuals']['subtitle']
        
        posts.append(newObj)
    
    return posts

In [153]:
# scrape page to get clapped posts
def getClaps(username, userId):
    print("Getting user claps...")
    
    url = MEDIUM + "@" + username + "/has-recommended?format=json"

    response = requests.get(url)
    response_dict = clean_json_response(response)
    
    payload = response_dict['payload']
    items = payload['streamItems']
    
    posts = parsePostsPayload(payload, userId) 
    
    filtered_posts = posts[:]
    for p in posts:
        if p['userId'] == userId:
            filtered_posts.remove(p)
    
    print('done')
    
    return filtered_posts

In [154]:
def getUserPosts(username, userId):

    print('Retrieving the latest posts...')

    post_ids = []
    
    url = MEDIUM + '@' + username + '/latest?format=json'
    response = requests.get(url)
    response_dict = clean_json_response(response)
    
    try:
        posts = response_dict['payload']['references']['Post']
    except:
        return []
    
    payload = response_dict['payload']
    posts = parsePostsPayload(payload, userId)
    
    print('done')
    
    return posts

In [155]:
(getClaps('RebeccaRennerFL', 'af97a6a078cf'))

Getting user claps...
done


[{'id': '4ccbfbb14314',
  'clapCount': 11,
  'wordCount': 1394,
  'readingTime': 5.960377358490566,
  'tags': ['Work', 'Work Life Balance', 'Anxiety', 'Advice'],
  'title': 'Ten Ways to Thrive in a Corporate Job if you have Anxiety',
  'userId': 'bb51e8f2c528',
  'time': 1542405250600,
  'subtitle': 'The corporate world can be intimidating, especially when you’re first starting out. Add to that a pounding heart or nagging stream of…'},
 {'id': 'e1b39c8c38ea',
  'clapCount': 1984,
  'wordCount': 1546,
  'readingTime': 5.833962264150943,
  'tags': ['Facebook', 'Social Media'],
  'title': 'The Definitive Tactical Guide to Quitting Facebook',
  'userId': 'eb358f417ed6',
  'time': 1541514661500,
  'subtitle': 'How to cover all your bases before leaving the site for good'},
 {'id': '7898ab6bc23e',
  'clapCount': 270,
  'wordCount': 2213,
  'readingTime': 9.050943396226414,
  'tags': ['Politics',
   'Elections',
   'Election 2018',
   'Donald Trump',
   'Equality'],
  'title': 'Yes, I Voted. 

In [263]:
def getUser(userId):
    url = "https://medium.com/_/api/users/" + userId + "/profile/stream?limit=25&source=followers"
    
    response = requests.get(url)
    j = clean_json_response(response)
    j = j['payload']
    
    f = j['streamItems'][1:]
    
    user = {}
    user['username'] = j['user']['username']
    user['userId'] = j['user']['userId']
    user['name'] = j['user']['name']
    user['tags'] = [tag['name'] for tag in j['userMeta']['authorTags']]
    user['followers'] = getFollowers(userId)
    user['following'] = getFollowing(userId)
    user['bio'] = j['user']['bio']
    user['followerCount'] = j['references']['SocialStats'][userId]['usersFollowedByCount']
    user['followCount'] = j['references']['SocialStats'][userId]['usersFollowedCount']
    user['postCount'] = j['userMeta']['numberOfPostsPublished']
    
    userPosts = getUserPosts(user['username'], userId) # list of post objects
    user['posts'] = [p['id'] for p in userPosts] # list of pospt ids
    
    userClaps = getClaps(user['username'], userId)
    user['recommends'] = [p['id'] for p in userClaps] # list of post objects that user has clapped on
    
    return user, userPosts+userClaps

In [264]:
startId = "af97a6a078cf"
startUser, posts = getUser(startId)

Fetching user followers...
total = 2277
Fetching user followers...
total = 2277
Retrieving the latest posts...
done
Getting user claps...
done


In [43]:
import requests

def get_list_of_latest_posts_ids(usernames):

    print('Retrieving the latest posts...')

    post_ids = []
    
    for username in usernames:
        url = MEDIUM + '/@' + username + '/latest?format=json'
        response = requests.get(url)
        response_dict = clean_json_response(response)
        
        try:
            posts = response_dict['payload']['references']['Post']
        except:
            posts = []

        if posts:
            for key in posts.keys():
                post_ids.append(posts[key]['id'])

    return post_ids


In [41]:
def get_post_responses(posts):

    print('Retrieving the post responses...')

    responses = []

    for post in posts:
        url = MEDIUM + '/_/api/posts/' + post + '/responses'
        response = requests.get(url)
        response_dict = clean_json_response(response)
        responses += response_dict['payload']['value']

    return responses

In [44]:
def check_if_high_recommends(response, recommend_min):
    if response['virtuals']['recommends'] >= recommend_min:
        return True

In [45]:
from datetime import datetime, timedelta

def check_if_recent(response):
    limit_date = datetime.now() - timedelta(days=30)
    creation_epoch_time = response['createdAt'] / 1000
    creation_date = datetime.fromtimestamp(creation_epoch_time)

    if creation_date >= limit_date:
        return True

In [52]:
def get_user_ids_from_responses(responses, recommend_min):

    print('Retrieving user IDs from the responses...')

    user_ids = []

    for response in responses:
        recent = check_if_recent(response)
        high = check_if_high_recommends(response, recommend_min)

        if recent or high:
            user_ids.append(response['creatorId'])

    return user_ids

In [ ]:
# End product:
# postId -> postObj
# userId -> userObj

# userObj -> name, username, followersId, followeesId, postsId, clapsId, categories, bio
# postObj -> author, responses, categories, title, time read, clapcount

In [333]:
# write header to file
f = csv.writer(open("users.csv", 'w'))
f.writerow(users[0].keys())

g = csv.writer(open("posts.csv", "w"))
g.writerow(posts[0].keys())

68

In [334]:
def writeUserToFile(filename, u): # append to file
    f = csv.writer(open(filename, "a"))
    f.writerow([u[k] for k in u.keys()])

def writePostToFile(filename, p):
    f = csv.writer(open(filename, "a"))
    for post in p:
        f.writerow([post[k] for k in post.keys()])    

In [360]:
# takes starting user, limit
# returns -> users, post-ids (to be scraped)
def scrapeUsers(startId, limit):
    users = []
    posts = []
    
    user, p = getUser(startId)
    
    userids = set([startId])
    postids = set([post['id'] for post in p])
    
    posts += p
    writePostToFile('posts.csv', posts)
    users.append(user)
    writeUserToFile('users.csv', user)
    
    queue = [user['userId']]
    
    while len(users) < limit:
        # current userr
        curUid = queue[0]
        queue.remove(curUid)
        
        if curUid == None: continue
        
        # fetch user object
        u, p = getUser(curUid)
        
        users.append(u)
        userids.add(curUid)
        
        # remove duplicates
        p = [post for post in p if post['id'] not in postids]

        writePostToFile('posts.csv', p)
        writeUserToFile('users.csv', u)

        posts += p
        postids.update([post['id'] for post in p])
        
        # add neighboring users to queue
        for uid in u['following']:
            if uid in userids: continue
            
            queue.append(uid)
    
    return users, posts

In [349]:
users, posts = scrapeUsers('af97a6a078cf', 7500)

Fetching user followers...
total = 2277


KeyboardInterrupt: 

In [352]:
uKeys = ['username', 'userId', 'name', 'tags', 'followers', 'following', 'bio', 'followerCount', 'followCount', 'postCount', 'posts', 'recommends']
pKeys = ["id", "clapCount", "wordCount", "readingTime", "tags", "title", "userId", "time", "subtitle"]

In [ ]:
users, posts = scrapeUsers('de9100e3b468', 600)

Fetching user followers...
total = 523
Fetching user following...
total = 552
Retrieving the latest posts...
done
Getting user claps...
done
Fetching user followers...
total = 523
Fetching user following...
total = 552
Retrieving the latest posts...
done
Getting user claps...
done
Fetching user followers...
total = 17
Fetching user following...
total = 18
Retrieving the latest posts...
Getting user claps...
done
Fetching user followers...
total = 12805
Fetching user following...
total = 2015
Retrieving the latest posts...
done
Getting user claps...
done
Fetching user followers...
total = 589
Fetching user following...
total = 154
Retrieving the latest posts...
done
Getting user claps...
done
Fetching user followers...
total = 6640
Fetching user following...
total = 2374
Retrieving the latest posts...
done
Getting user claps...
done
Fetching user followers...
total = 199
Fetching user following...
total = 295
Retrieving the latest posts...
Getting user claps...
done
Fetching user follow

Retrieving the latest posts...
done
Getting user claps...
done
Fetching user followers...
total = 51
Fetching user following...
total = 131
Retrieving the latest posts...
Getting user claps...
done
Fetching user followers...
total = 79
Fetching user following...
total = 53
Retrieving the latest posts...
Getting user claps...
done
Fetching user followers...
total = 389
Fetching user following...
total = 336
Retrieving the latest posts...
Getting user claps...
done
Fetching user followers...
total = 32713
Fetching user following...
total = 412
Retrieving the latest posts...
done
Getting user claps...
done
Fetching user followers...
total = 962
Fetching user following...
total = 857
